In [ ]:
import pandas as pd
import numpy as np

from sklearn import model_selection
from sklearn import metrics

import lightgbm as lgb

In [ ]:
df_train = pd.read_csv('/kaggle/input/playground-series-s3e14/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s3e14/test.csv')

In [ ]:
df_train = df_train.drop(['id', 'MinOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange',
                          'AverageOfLowerTRange','AverageRainingDays'] , axis=1)
df_test = df_test.drop(['id', 'MinOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange',
                            'AverageOfLowerTRange','AverageRainingDays'] , axis=1)

In [ ]:
X = df_train.drop(['yield'], axis=1)
y = df_train['yield']
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
reg_train = lgb.Dataset(x_train, y_train)
reg_test = lgb.Dataset(x_valid, y_valid, reference=reg_train)


In [ ]:
params = {'n_jobs': -1,
        'learning_rate': 0.04,
        'max_bin': 1000,
        'colsample_bytree': .8,
        'subsample': .7, 
        'bagging_freq': 1,
        'objective': 'regression_l1',
        'metric': 'mae',
        'early_stopping_rounds': 200,
        'verbosity': -1}

model = lgb.train(params, reg_train, num_boost_round=10000, valid_sets=[reg_train, reg_test], verbose_eval=1000)

In [ ]:
submit_params = {'n_jobs': -1,
        'learning_rate': 0.04,
        'max_bin': 1000,
        'colsample_bytree': .8,
        'subsample': .7, 
        'bagging_freq': 1,
        'objective': 'regression_l1',
        'metric': 'mae',
        'verbosity': -1}
submit_train = lgb.Dataset(X, y)
model = lgb.train(submit_params, submit_train, num_boost_round=10000, verbose_eval=1000)

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/playground-series-s3e14/sample_submission.csv')
pred = model.predict(df_test)
sample_submission['yield'] = pred
sample_submission.to_csv('submission.csv', index=False)
